# Demo for SPEL using LakeTemperature


In [ ]:
%tb
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Built-in modules
import re # Regular expressions
import importlib # MUST BE USED TO RELOAD MODULES

# Importing modules for SPEL functions 
import utilityFunctions as uf
import edit_files as ef 
import fortran_modules as fm 
from analyze_subroutines import Subroutine
# mod_config : system configuration and static variables. 
# Where E3SM is stored, where unit tests are stored, etc.
from mod_config import default_mods, unittests_dir, scripts_dir, spel_mods_dir
from mod_config import ELM_SRC, spel_output_dir, _bc


SPEL needs a "casename" (what your unit-test is called) and a list of subroutines to create a unit-test for "sub_name_list".  For this demo, the sub_name_list only has "LakeTemperature" in it.


In [ ]:
# Define Unit Test parameters 
casename = "LakeTemp"  # Name of the test case
case_dir = unittests_dir + casename # Directory to store the test case

# List of subroutines to be analyzed
sub_name_list = ["LakeTemperature"]

# variables needed for Unit Test
main_sub_dict = {}  # Dictionary to store all Subroutines in files needed for Unit Test
subroutines = {k.lower():[] for k in sub_name_list} # Dictionary for User Specified Subroutines

1st step for SPEL is determine which modules are needed for LakeTemperature and edit out the I/O, MPI, and other unneccessary modules. This must be somewhat tailored to ELM.

Currently if I module is not present in the "components/elm/src/" (ELM_SRC) nor in the "share/utils" directories (SHR_SRC), then the module and any dependency on it is automatically removed.

In [ ]:
 # List to hold all the modules needed for the unit test
needed_mods = [] 
for s in sub_name_list:
    # Get general info of the subroutine
    subroutines[s] = Subroutine(s,calltree=['elm_drv'])

    # Process by removing certain modules and syntax
    # so that a standalone unit test can be compiled.
    fn = subroutines[s].filepath
    mod_dict, file_list = ef.process_for_unit_test(fname=fn,case_dir=case_dir,
                            mods=needed_mods,required_mods=default_mods, 
                            main_sub_dict=main_sub_dict,
                            overwrite=False,verbose=False)

In [ ]:
# for mod in mod_dict.values():
#     mod.display_info()

type_dict = {}
for modname, mod in mod_dict.items():
    for utype, dtype in mod.defined_types.items():
        type_dict[utype] = dtype

SPEL can print out a module tree showing how the modules are linked for the unit-test subroutines
- The full tree can be pretty difficult to parse, having a cutoff depth is recommended.
- Every module uses 'shr_kind_mod' so that could be suppressed as well.

In [ ]:
importlib.reload(fm)
modtree = fm.print_spel_module_dependencies(mod_dict=mod_dict,subs=subroutines)

arrow = "-->"
cutoff_depth = 10 # Only print modules up to this depth
suppress_mod_list = ['shr_kind_mod']
for m in modtree:
    depth = m['depth']
    modname = m['module']
    if(modname in suppress_mod_list):
        continue
    if(depth == 1):
        print(_bc.HEADER + arrow*depth + modname + _bc.ENDC)
    elif(depth <= cutoff_depth):
        print( arrow*depth + modname)

In [ ]:
test_mod = mod_dict['laketemperaturemod']
test_mod.display_info()

test_mod = mod_dict['ch4mod']
test_mod.display_info()

In [ ]:
for f in file_list:
    base_fn = f.split('/')[-1]
    print(base_fn)

In [ ]:
# for sub in main_sub_dict.values():
#     print(sub.name)
sub = main_sub_dict['laketemperature']
sub.printSubroutineInfo()

In [ ]:
sub.LocalVariables['arrays']

In [ ]:
for s in sub_name_list:
    # Parsing means getting info on the variables read and written
    # to by the subroutine and any of its callees
    subroutines[s].parse_subroutine(dtype_dict=type_dict,
                                    main_sub_dict=main_sub_dict,verbose=True)

In [ ]:
subroutines['LakeTemperature'].child_Subroutine.keys()


In [ ]:
var = "snl"
dummy = "jtop( bounds%begc: bounds%endc, snl(c),nasdk+f)"
import re
regex_indices = re.compile(r'(?<=\()(.+)(?=\))')
ind = regex_indices.search(dummy).group(0)
print(ind)
regex_var = re.compile(r'({})'.format(var))
print(regex_var.search(ind).group(0))
print(regex_indices.sub('',dummy))